In [83]:
from collections import Counter
from operator import add 
import csv
import string
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer


global Path
Path = "/home/ethan/pythonwork/ipynotebook/hw2/"

# read file
with open( Path + "News_Final.csv" , 'r', encoding= 'utf8') as news:
    data = csv.reader(news, delimiter = ",")
    listdata = list(data)

# change to rdd    
subDataset = sc.parallelize(listdata)

# remove header
header = subDataset.first()
subData = subDataset.filter(lambda x: x!=header)

# outputfile
output_file = "hw2_4_"


#------------------- Title  -------------------------

print("Start Title processing")

count_topic_title = subData.map(lambda x: (x[4], x[1]) ).reduceByKey(lambda a, b: a+b)
key_topic = count_topic_title.map(lambda x: x[0])

for t in range(0, count_topic_title.count()):
    topic = key_topic.collect()[t]
    
    print("Start find all row_data in", topic)
    row_data = subData.filter(lambda x: x[4] == topic)\
                         .map( lambda x: re.split(" ", x[1]))\
                         .map( lambda x : [ re.sub("^[^a-zA-Z0-9\$]","", i ) for i in x ] )\
                         .map( lambda x : [ re.sub("[^a-zA-Z0-9]$","", i ) for i in x ] )

    print("Start find top-100 TF in", topic)
    value_topic = row_data.map( lambda x : [ (i,1) for i in x[0:] if(i!='...') if(i!='') ] )\
                          .flatMap(lambda list: list)\
                          .reduceByKey(lambda a, b: a+b).sortBy(lambda x: -x[1]).map(lambda x: x[0])               
    top_100 = value_topic.take(100)
#     top_100_word = top_100.map(lambda x: x[0])
    
    
    # initial 100x100 matrix                    
    Matrix = np.zeros((100,100),int)
    
    topic_rowdata_count = row_data.count()
    topic_top100_count = len(top_100)
    topic_rowdata = row_data.collect()
#     topic_top100 = top_100.collect()
    
    print("Start processing co-occurrence in ", topic)
    for i in range(0, topic_rowdata_count):
        for j in range(0, topic_top100_count):
            if ((top_100[j]) in (topic_rowdata[i])) :
                for k in range(0, topic_top100_count):
                    if (top_100[k] in topic_rowdata[i] and j!=k ):
                        Matrix[j][k] = Matrix[j][k] + 1
    
    
    print("Start output in ", topic)              
    with open(output_file + topic + "_intitle_pertopic.txt" , 'a') as file:
        file.write("  ")
        for i in range(0, topic_top100_count):
            file.write(top_100[i] + " ")
        file.write("\n")
        
        for j in range(0, topic_top100_count):
            file.write(top_100[j] + " ")
            
            for k in range(0, topic_top100_count):
                file.write(str(Matrix[j][k]) + " ")
            file.write("\n")

print("Title output Success!")

#------------------- headline  -------------------------

print("Start headline processing")

count_topic_title = subData.map(lambda x: (x[4], x[2]) ).reduceByKey(lambda a, b: a+b)
key_topic = count_topic_title.map(lambda x: x[0])

for t in range(0, count_topic_title.count()):
    topic = key_topic.collect()[t]
    
    print("Start find all row_data in", topic)
    row_data = subData.filter(lambda x: x[4] == topic)\
                         .map( lambda x: re.split(" ", x[1]))\
                         .map( lambda x : [ re.sub("^[^a-zA-Z0-9\$]","", i ) for i in x ] )\
                         .map( lambda x : [ re.sub("[^a-zA-Z0-9]$","", i ) for i in x ] )

    print("Start find top-100 TF in", topic)
    value_topic = row_data.map( lambda x : [ (i,1) for i in x[0:] if(i!='...') if(i!='') ] )\
                          .flatMap(lambda list: list)\
                          .reduceByKey(lambda a, b: a+b).sortBy(lambda x: -x[1]).map(lambda x: x[0])               
    top_100 = value_topic.take(100)
#     top_100_word = top_100.map(lambda x: x[0])
    
    
    # initial 100x100 matrix                    
    Matrix = np.zeros((100,100),int)
    
    topic_rowdata_count = row_data.count()
    topic_top100_count = len(top_100)
    topic_rowdata = row_data.collect()
#     topic_top100 = top_100.collect()
    
    print("Start processing co-occurrence in ", topic)
    for i in range(0, topic_rowdata_count):
        for j in range(0, topic_top100_count):
            if ((top_100[j]) in (topic_rowdata[i])) :
                for k in range(0, topic_top100_count):
                    if (top_100[k] in topic_rowdata[i] and j!=k ):
                        Matrix[j][k] = Matrix[j][k] + 1
    
    
    print("Start output in ", topic)              
    with open(output_file + topic + "_inheadline_pertopic.txt" , 'a') as file:
        file.write("  ")
        for i in range(0, topic_top100_count):
            file.write(top_100[i] + " ")
        file.write("\n")
        
        for j in range(0, topic_top100_count):
            file.write(top_100[j] + " ")
            
            for k in range(0, topic_top100_count):
                file.write(str(Matrix[j][k]) + " ")
            file.write("\n")

print("Headline output Success!")
print("Finished!")

Start Title processing
Start find all row_data in economy
Start find top-100 TF in economy
Start processing co-occurrence in  economy
Start output in  economy
Start find all row_data in microsoft
Start find top-100 TF in microsoft
Start processing co-occurrence in  microsoft
Start output in  microsoft
Start find all row_data in palestine
Start find top-100 TF in palestine
Start processing co-occurrence in  palestine
Start output in  palestine
Start find all row_data in obama
Start find top-100 TF in obama
Start processing co-occurrence in  obama
Start output in  obama
Title output Success!
Finished!
